In [ ]:
%pip install fashion-clip
%pip install faiss-cpu

In [9]:
# Generar un pandas con el dataset inditextech_hackupc_challenge

import pandas as pd
import numpy as np

# Cargamos el dataset
df = pd.read_csv('inditextech_hackupc_challenge_images.csv')

batch_size = 1000
start = 0
df_batch = df[start:start+batch_size]

In [8]:
"""
import requests
import numpy as np
import pandas as pd
from PIL import Image
import io

# Suponiendo que tienes un DataFrame llamado df con una columna llamada 'IMAGE_VERSION_3' que contiene las URLs de las imágenes

# Función para leer la imagen desde la URL y convertirla en un array de numpy
def read_image(url):
    try:
        response = requests.get(url)
        image_bytes = io.BytesIO(response.content)
        image = Image.open(image_bytes)
    except:
        return None 
    return np.array(image)

# Generamos el pandas
df_batch['id'] = df_batch.index
# leer la imagen de la tercera url de la fila y convertirla a numpy array
df_batch['image'] = df_batch['IMAGE_VERSION_3'].apply(lambda x: read_image(x) if pd.notnull(x) else None)
df_batch
"""

C:\Users\ralva\AppData\Local\Temp\ipykernel_23636\1645700350.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_batch['id'] = df_batch.index
C:\Users\ralva\AppData\Local\Temp\ipykernel_23636\1645700350.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_batch['image'] = df_batch['IMAGE_VERSION_3'].apply(lambda x: read_image(x) if pd.notnull(x) else None)


,IMAGE_VERSION_1,IMAGE_VERSION_2,IMAGE_VERSION_3,id,image
0,https://static.zara.net/photos///2024/V/0/3/p/...,https://static.zara.net/photos///2024/V/0/3/p/...,https://static.zara.net/photos///2024/V/0/3/p/...,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,https://static.zara.net/photos///2024/V/0/3/p/...,https://static.zara.net/photos///2024/V/0/3/p/...,https://static.zara.net/photos///2024/V/0/3/p/...,1,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
2,https://static.zara.net/photos///2024/V/0/1/p/...,https://static.zara.net/photos///2024/V/0/1/p/...,https://static.zara.net/photos///2024/V/0/1/p/...,2,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
3,https://static.zara.net/photos///2024/V/0/2/p/...,https://static.zara.net/photos///2024/V/0/2/p/...,https://static.zara.net/photos///2024/V/0/2/p/...,3,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
4,https://static.zara.net/photos///2024/V/0/2/p/...,https://static.zara.net/photos///2024/V/0/2/p/...,https://static.zara.net/photos///2024/V/0/2/p/...,4,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
...,...,...,...,...,...
95,https://static.zara.net/photos///2024/V/1/1/p/...,https://static.zara.net/photos///2024/V/1/1/p/...,https://static.zara.net/photos///2024/V/1/1/p/...,95,"[[[238, 238, 236], [237, 237, 235], [237, 237,..."
96,https://static.zara.net/photos///2024/V/0/1/p/...,https://static.zara.net/photos///2024/V/0/1/p/...,https://static.zara.net/photos///2024/V/0/1/p/...,96,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
97,https://static.zara.net/photos///2024/V/0/1/p/...,https://static.zara.net/photos///2024/V/0/1/p/...,https://static.zara.net/photos///2024/V/0/1/p/...,97,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
98,https://static.zara.net/photos///2024/V/0/2/p/...,https://static.zara.net/photos///2024/V/0/2/p/...,https://static.zara.net/photos///2024/V/0/2/p/...,98,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."


In [75]:
import os
import pandas as pd
import requests
from random import randint
import io
import time

# Suponiendo que df_batch ya está definido
inditex_images = pd.DataFrame(columns=['id', 'path_to_image'])

# Funcion para detectar que la respuesta es una imagenç
def is_image(response):
    image_formats = ("image/png", "image/jpeg", "image/jpg")
    return response.headers["content-type"] in image_formats

# Añadir los items al DataFrame inditex_images
for index, row in df_batch.iterrows():
    url = row['IMAGE_VERSION_3']
    if pd.notna(url):
        try:
            response = requests.get(url)
            response.raise_for_status() # Lanzar una excepción si la respuesta no es 200
            # Escribir la imagen en un archivo
            filename = f'inditex_images/{index}.jpg'
            with open(filename, 'wb') as file:
                file.write(response.content)
                
            # Agregar la ruta de la imagen al DataFrame
            new_row = pd.DataFrame({'id': [index], 'path_to_image': [filename]})
            inditex_images = pd.concat([inditex_images, new_row], ignore_index=True)
            
        except Exception as e:
            print(f"No se pudo descargar o guardar la imagen {index} debido a: {e}")
            time.sleep(randint(0, 10)/10)  # Esperar un segundo antes de intentar la siguiente descarga
    else:
        print(f'No hay URL válida para la imagen {index}')

# Mostrar el DataFrame actualizado
print(inditex_images)


No hay URL válida para la imagen 7
No hay URL válida para la imagen 9
No hay URL válida para la imagen 14
No hay URL válida para la imagen 18
No hay URL válida para la imagen 20
No hay URL válida para la imagen 28
No hay URL válida para la imagen 42
No hay URL válida para la imagen 54
No hay URL válida para la imagen 55
No hay URL válida para la imagen 62
No hay URL válida para la imagen 63
No hay URL válida para la imagen 65
No hay URL válida para la imagen 66
No se pudo descargar o guardar la imagen 69 debido a: 403 Client Error: Forbidden for url: https://static.zara.net/photos///2024/V/0/2/p/4080/017/802/2/w/4000/4080017802_3_1_1.jpg?ts=1701098484553
No hay URL válida para la imagen 70
No hay URL válida para la imagen 71
No hay URL válida para la imagen 72
No hay URL válida para la imagen 102
No hay URL válida para la imagen 119
No hay URL válida para la imagen 122
No hay URL válida para la imagen 136
No hay URL válida para la imagen 137
No hay URL válida para la imagen 140
No hay 

In [29]:
Image.open(io.BytesIO(requests.get(df_batch['IMAGE_VERSION_3'][7]).content)).save('test.jpg')

MissingSchema: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?

# Embedings

___

In [13]:
# Cargar el pandas desde un archivo pkl
df = pd.read_pickle('inditextech_hackupc_challenge_images.pkl')

,IMAGE_VERSION_1,IMAGE_VERSION_2,IMAGE_VERSION_3,id,image
0,https://static.zara.net/photos///2024/V/0/3/p/...,https://static.zara.net/photos///2024/V/0/3/p/...,https://static.zara.net/photos///2024/V/0/3/p/...,0,None
1,https://static.zara.net/photos///2024/V/0/3/p/...,https://static.zara.net/photos///2024/V/0/3/p/...,https://static.zara.net/photos///2024/V/0/3/p/...,1,None
2,https://static.zara.net/photos///2024/V/0/1/p/...,https://static.zara.net/photos///2024/V/0/1/p/...,https://static.zara.net/photos///2024/V/0/1/p/...,2,None
3,https://static.zara.net/photos///2024/V/0/2/p/...,https://static.zara.net/photos///2024/V/0/2/p/...,https://static.zara.net/photos///2024/V/0/2/p/...,3,None
4,https://static.zara.net/photos///2024/V/0/2/p/...,https://static.zara.net/photos///2024/V/0/2/p/...,https://static.zara.net/photos///2024/V/0/2/p/...,4,None
...,...,...,...,...,...
49313,https://static.zara.net/photos///2021/I/4/1/p/...,https://static.zara.net/photos///2021/I/4/1/p/...,NaN,49313,None
49314,https://static.zara.net/photos///2024/V/0/3/p/...,https://static.zara.net/photos///2024/V/0/3/p/...,https://static.zara.net/photos///2024/V/0/3/p/...,49314,None
49315,https://static.zara.net/photos///2024/V/1/3/p/...,https://static.zara.net/photos///2024/V/1/3/p/...,https://static.zara.net/photos///2024/V/1/3/p/...,49315,None
49316,https://static.zara.net/photos///2024/V/0/3/p/...,https://static.zara.net/photos///2024/V/0/3/p/...,https://static.zara.net/photos///2024/V/0/3/p/...,49316,None


In [61]:
# Generar un wordembeddings con el dataset
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# Tenemos un pandas con una imagen pasada a numpy array en la columna 'image'

# Función para generar un wordembeddings con las imágenes
def generate_wordembeddings(df):
    # Cargar el modelo preentrenado de word2vec
    model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
    
    # Generar el wordembeddings
    wordembeddings = []
    for i in range(len(df)):
        image = df['image'][i]
        if image is not None:
            wordembedding = np.mean([model[word] for word in image.flatten() if word in model], axis=0)
        else:
            wordembedding = None
        wordembeddings.append(wordembedding)
    return wordembeddings

KeyboardInterrupt: 